## 1_でクロールしたpicleファイルをロードし、一つにまとめて、解析に使いやすいようにデータを整形したうえで再度pickleファイルで保存
- ファイルの読み込み元：'./crawledData/race_result　以下
- ファイルの保存先：'./arrangedData/　以下

In [14]:
import glob
import pandas as pd

#### レース情報をまとめたpicleファイルを読み込んで一つにまとめる

In [15]:
# ファイル内容確認用
all_files = glob.glob('../../data/crawledData/race_result/*.pkl')

# その日のレース情報をまとめたdfを要素にもつリスト
race_df_list = [pd.read_pickle(file) for file in all_files]
# concatして一つのdfにまとめる
race_df = pd.concat(race_df_list)
race_df

racer_id racer_class racer_name num_false_start  \
date       venue raceNumber 枠                                                   
2020-11-03 桐　生   1          1     4851          A1     関　　　浩哉              F0   
                            2     4362          A1     土屋　　智則              F0   
                            3     4080          A1     山崎　　哲司              F0   
                            4     3554          A1     仲口　　博崇              F0   
                            5     4504          A1     前田　　将太              F0   
...                                ...         ...        ...             ...   
2020-11-26 福　岡   12         2     4816          A1     村松　　修二              F0   
                            3     4737          A1     竹井　　貴史              F1   
                            4     4643          A2     近江　　翔吾              F0   
                            5     4802          A1     中田　　達也              F0   
                            6     4960          A2     黒野　　元基              F0   

                              num_late_start motorNo モーター2連率 モーター3連率 boatNo  \
date       venue raceNumber 枠                                                 
2020-11-03 桐　生   1          1             L0      73   33.33   46.20     61   
                            2             L0      20   43.98   61.45     14   
                            3             L0      24   38.20   61.24     40   
                            4             L0      23   34.39   47.51     59   
                            5             L0      59   47.73   63.64     70   
...                                      ...     ...     ...     ...    ...   
2020-11-26 福　岡   12         2             L0      70   25.97   48.05    135   
                            3             L0      47   26.25   38.75    116   
                            4             L0      57   24.68   40.26    120   
                            5             L0      67   31.46   50.56    174   
                            6             L0      14   42.03   60.87    106   

                               ボート2連率  ... wave_height weight exhibitionTime  \
date       venue raceNumber 枠          ...                                     
2020-11-03 桐　生   1          1   28.49  ...           1   49.5           6.74   
                            2   35.91  ...           1   51.1           6.67   
                            3   35.71  ...           1   51.0           6.70   
                            4   41.01  ...           1   51.4           6.76   
                            5   34.64  ...           1   52.8           6.82   
...                               ...  ...         ...    ...            ...   
2020-11-26 福　岡   12         2   46.97  ...           2   49.5           6.86   
                            3   36.84  ...           2   51.5           6.78   
                            4   30.26  ...           2   51.5           6.86   
                            5   32.89  ...           2   51.5           6.81   
                            6   37.35  ...           2   52.3           6.79   

                               tilt 着順     タイム 進入コース start_time  \
date       venue raceNumber 枠                                     
2020-11-03 桐　生   1          1  -0.5  １  1'49"8   1.0       0.16   
                            2  -0.5  ４  1'53"7   2.0       0.25   
                            3  -0.5  ３  1'53"0   5.0       0.09   
                            4  -0.5  ２  1'51"5   3.0       0.22   
                            5  -0.5  ５  1'53"7   6.0       0.18   
...                             ... ..     ...   ...        ...   
2020-11-26 福　岡   12         2  -0.5  １  1'49"4   2.0       0.06   
                            3  -0.5  ３  1'51"6   3.0       0.11   
                            4  -0.5  ２  1'50"0   4.0       0.09   
                            5   0.0  ５           5.0       0.11   
                            6  -0.5  ６           6.0       0.11   

                              exhib

#### 解析に使いやすいようにデータを整形する

In [17]:
from datetime import datetime as dt
import numpy as np

def arrange_exhibition_start_time(exhibition_start_time):
    # flying, lateを反映する変数を作成
    flying = False
    late = False
    if str(exhibition_start_time)[0] == "F":
        flying = True
        try:
            time = float(str(exhibition_start_time)[1:])
        except ValueError:
            time = np.nan
    elif str(exhibition_start_time)[0] == "L":
        late = True
        time = np.nan
    else:
        time = float(exhibition_start_time)
    
    return pd.Series([flying, late, time])


race_df_arranged = race_df.copy()

# それぞれの列のデータタイプの変更など
race_df_arranged["boatNo"] = race_df_arranged["boatNo"].astype("int")
race_df_arranged["motorNo"] = race_df_arranged["motorNo"].astype("int")
race_df_arranged["num_false_start"] = race_df_arranged["num_false_start"].map(lambda x: int(str(x)[1]))
race_df_arranged["num_late_start"] = race_df_arranged["num_late_start"].map(lambda x: int(str(x)[1]))
race_df_arranged["racer_id"] = race_df_arranged["racer_id"].astype("int")
race_df_arranged["ボート2連率"] = race_df_arranged["ボート2連率"].astype("float64")
race_df_arranged["ボート3連率"] = race_df_arranged["ボート3連率"].astype("float64")
race_df_arranged["racer_class"] = race_df_arranged["racer_class"].map({"A1": 1, "A2": 2, "B1": 3, "B2": 4})
race_df_arranged["モーター2連率"] = race_df_arranged["モーター2連率"].astype("float64")
race_df_arranged["モーター3連率"] = race_df_arranged["モーター3連率"].astype("float64")
race_df_arranged["tilt"] = race_df_arranged["tilt"].astype("float64")
race_df_arranged["weight"] = race_df_arranged["weight"].astype("float64")

# exhibition start timeの列を、exhibition_fault列と、late列と、time列に分ける
race_df_arranged[["exhibition_flying", "exhibition_late", "exhibition_st"]] = race_df_arranged["exhibition_start_time"].apply(arrange_exhibition_start_time)
race_df_arranged.drop("exhibition_start_time", axis=1, inplace=True)

# weatherをdummy変数に変換
dummy_df_weather = pd.get_dummies(race_df_arranged["weather"], prefix='weather', drop_first=True)
race_df_arranged = pd.concat([race_df_arranged, dummy_df_weather], axis=1)

# 何らかの事情でデータから抜いたほうがいいレースを集める
race_df_del_list = []
# 転覆等があり、着順が6までつけられなかったレースを削除. 転覆になった選手の行だけではなくそのレースごと列を削除する
race_df_del_1 = race_df_arranged[race_df_arranged["着順"].astype(str).str.contains("\D")]
race_df_del_1.reset_index(inplace=True)
race_df_del_list.append(race_df_del_1)

# それぞれvalueに''を含むレースを削除
may_have_blank_list = [
    "exhibitionTime", "temperature", "water_temperature",
    "wave_height", "wind_speed", "exhibition_cource"
]

# 正規表現に引っ掛けるためにstr型にしておく
race_df_arranged["exhibition_cource"] = race_df_arranged["exhibition_cource"].astype("str")

for column in may_have_blank_list:
    race_df_del_ = race_df_arranged[~race_df_arranged[column].str.match("\d.*")]
    race_df_del_.reset_index(inplace=True)
    race_df_del_list.append(race_df_del_)

# Nanを含む列があるレースを削除
race_df_del_nan = race_df_arranged[race_df_arranged.isnull().any(axis=1)]
race_df_del_nan.reset_index(inplace=True)
race_df_del_list.append(race_df_del_nan)

# 消したいレースの開催地・場所・レース番号のリストを作成
remove_race_list_nested = [race_df_del[["date", "venue", "raceNumber"]].values.tolist() for race_df_del in race_df_del_list]
remove_race_list =  [item for l in remove_race_list_nested for item in l]

remove_index_list = list(set([tuple(race) for race in remove_race_list]))
race_df_arranged.drop(index=remove_index_list, inplace=True)

# 着順のカラムをinterger型に変換
race_df_arranged["着順"] = race_df_arranged["着順"].astype("int")

race_df_arranged["exhibitionTime"] = race_df_arranged["exhibitionTime"].astype("float64")
race_df_arranged["temperature"] = race_df_arranged["temperature"].astype("float64")
race_df_arranged["water_temperature"] = race_df_arranged["water_temperature"].astype("float64")
race_df_arranged["wave_height"] = race_df_arranged["wave_height"].astype("int")
race_df_arranged["wind_speed"] = race_df_arranged["wind_speed"].astype("int")
race_df_arranged["exhibition_cource"] = race_df_arranged["exhibition_cource"].astype("float64")

# multiindexは扱いがめんどくさいので全部カラムにする
race_df_arranged.reset_index(inplace=True)

# date列をdatetim型に変換
race_df_arranged["date"] = race_df_arranged["date"].map(lambda x: dt.strptime(x, '%Y-%m-%d'))

# dateとraceNumberでsort
race_df_arranged["raceNumber"] = race_df_arranged["raceNumber"].astype("float")
race_df_arranged.sort_values(["date", "venue", "raceNumber", "枠"], inplace=True)

race_df_arranged

/Users/grice/opt/anaconda3/envs/boatrace2/lib/python3.8/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


,date,venue,raceNumber,枠,racer_id,racer_class,racer_name,num_false_start,num_late_start,motorNo,...,進入コース,start_time,exhibition_cource,exhibition_flying,exhibition_late,exhibition_st,weather_晴,weather_曇り,weather_雨,weather_雪
16680,2020-08-01,びわこ,1.0,1,3783,1,瓜生 正義,0,0,46,...,1.0,0.06,1.0,False,False,0.13,1,0,0,0
16681,2020-08-01,びわこ,1.0,2,4580,1,長谷川 雅和,0,0,54,...,2.0,0.13,2.0,False,False,0.06,1,0,0,0
16682,2020-08-01,びわこ,1.0,3,4261,1,岡 祐臣,0,0,41,...,3.0,0.18,3.0,False,False,0.24,1,0,0,0
16683,2020-08-01,びわこ,1.0,4,4064,1,原田 篤志,0,0,60,...,4.0,0.27,4.0,True,False,0.07,1,0,0,0
16684,2020-08-01,びわこ,1.0,5,4239,1,竹田 辰也,0,0,51,...,5.0,0.24,5.0,True,False,0.04,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28891,2021-04-30,鳴 門,12.0,2,3070,2,山室 展弘,0,0,12,...,2.0,0.14,2.0,False,False,0.18,1,0,0,0
28892,2021-04-30,鳴 門,12.0,3,3919,1,村上 純,1,0,11,...,3.0,0.18,3.0,False,False,0.11,1,0,0,0
28893,2021-04-30,鳴 門,12.0,4,3516,3,吉原 聖人,0,0,14,...,4.0,0.19,4.0,False,False,0.10,1,0,0,0
28894,2021-04-30,鳴 門,12.0,5,3333,1,丸尾 義孝,1,0,25,...,5.0,0.26,5.0,False,False,0.07,1,0,0,0


In [18]:
# 統計量の算出を行うためのdf: pickleファイルで保存
race_df_arranged.to_pickle('../../data/arrangedData/data_formatted_for_statistic.pkl')

# 予測の際にそのままinputにすることができない列を削除
race_df_arranged.drop(["boatNo", "motorNo", "racer_id", "racer_name", "weather", "タイム", "start_time", "進入コース"], axis=1, inplace=True)

# 予測用のdf: pickleファイルで保存
race_df_arranged.to_pickle('../../data/arrangedData/data_formatted_1.pkl')
race_df_arranged

,date,venue,raceNumber,枠,racer_class,num_false_start,num_late_start,モーター2連率,モーター3連率,ボート2連率,...,tilt,着順,exhibition_cource,exhibition_flying,exhibition_late,exhibition_st,weather_晴,weather_曇り,weather_雨,weather_雪
16680,2020-08-01,びわこ,1.0,1,1,0,0,22.50,47.50,32.35,...,-0.5,1,1.0,False,False,0.13,1,0,0,0
16681,2020-08-01,びわこ,1.0,2,1,0,0,34.15,58.54,25.00,...,-0.5,2,2.0,False,False,0.06,1,0,0,0
16682,2020-08-01,びわこ,1.0,3,1,0,0,50.00,70.45,42.86,...,-0.5,4,3.0,False,False,0.24,1,0,0,0
16683,2020-08-01,びわこ,1.0,4,1,0,0,47.62,78.57,42.42,...,-0.5,3,4.0,True,False,0.07,1,0,0,0
16684,2020-08-01,びわこ,1.0,5,1,0,0,53.49,67.44,31.03,...,-0.5,5,5.0,True,False,0.04,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28891,2021-04-30,鳴 門,12.0,2,2,0,0,12.50,25.00,88.89,...,-0.5,2,2.0,False,False,0.18,1,0,0,0
28892,2021-04-30,鳴 門,12.0,3,1,1,0,70.00,90.00,12.50,...,-0.5,6,3.0,False,False,0.11,1,0,0,0
28893,2021-04-30,鳴 門,12.0,4,3,0,0,12.50,12.50,0.00,...,-0.5,3,4.0,False,False,0.10,1,0,0,0
28894,2021-04-30,鳴 門,12.0,5,1,1,0,20.00,20.00,33.33,...,-0.5,5,5.0,False,False,0.07,1,0,0,0
